Based on some arguments we made in the paper, for a long, weak pulse, the expansion coefficient of the ground state should go like
$$ c_{1s}(t) = e^{-\gamma \, \int_{-\infty}^t \mathcal{E}^2(t)} $$
where $\kappa$ is a constant that depends on an integral over the kernel.

If this is the case, we should expect the final probability to be in the $1s$ state to be
$$ P_{1s}(\infty) = e^{-2 \gamma H / \epsilon_0 c} $$
where $H$ is the fluence of the pulse.
$\gamma$ is $\approx 1$, hopefully.

In [1]:
def make_scans_with_overlay(job_processor):
    for line_parameter, scan_parameter, plot_parameters in job_processor._select_two_parameters():
        try:
            line_parameter_set, scan_parameter_set = job_processor.parameter_set(line_parameter), job_processor.parameter_set(scan_parameter)
        except AttributeError:  # parameter is not tracked
            continue

        if any((len(scan_parameter_set) < 10,  # not enough x data, or too many lines
                len(line_parameter_set) > 8,)):
            continue

        plot_parameter_to_set = job_processor._get_parameters_to_parameter_sets(plot_parameters)
        if len(plot_parameter_to_set) == 0:  # this sometimes happens on heatmap jobs?
            continue

        plot_parameters = tuple(plot_parameter_to_set.keys())
        plot_parameter_value_groups = (plot_parameter_values for plot_parameter_values in itertools.product(*plot_parameter_to_set.values()))

        for plot_parameter_values in plot_parameter_value_groups:
            for ionization_metric in job_processor.ionization_metrics:
                base_selector = dict(zip(plot_parameters, plot_parameter_values))
                x_data, y_data, line_labels = job_processor._get_lines(line_parameter, scan_parameter, ionization_metric, base_selector)
                if any(x[0] == x[-1] for x in x_data):
                    continue

                str_for_filename, str_for_plot = job_processor._get_plot_parameter_strings(plot_parameters, plot_parameter_values)
                filename = '__'.join((
                    job_processor.name,
                    '1d',
                    ionization_metric,
                    str_for_filename,
                    f'scanning_{scan_parameter}_grouped_by_{line_parameter}',
                ))
                title = job_processor.name + '\n' + str_for_plot

                for log_x, log_y in itertools.product((True, False), repeat = 2):
                    if any((log_x and scan_parameter == 'phase',
                            log_x and any(np.any(x <= 0) for x in x_data),
                            log_y and any(np.any(y <= 0) for y in y_data),)):
                        continue

                    if log_y:
                        y_lower_limit = None
                    else:
                        y_lower_limit = 0

                    log_str = format_log_str(
                        (log_x, log_y),
                        ('X', 'Y'),
                    )

                    si.vis.xxyy_plot(
                        filename + log_str,
                        x_data,
                        y_data,
                        line_labels = line_labels,
                        x_label = fr'${PARAMETER_TO_SYMBOL[scan_parameter]}$',
                        x_unit = PARAMETER_TO_UNIT_NAME[scan_parameter],
                        x_log_axis = log_x,
                        y_label = format_ionization_metric_name(ionization_metric),
                        y_lower_limit = y_lower_limit,
                        y_upper_limit = 1,
                        y_log_axis = log_y,
                        title = title,
                        legend_on_right = True if len(line_labels) > 5 else False,
                        target_dir = job_processor.summaries_dir,
                        img_format = 'png',
                    )
